In [10]:
import pandas as pd

chunksize = 50000
chunks = pd.read_csv('ETL_Final.csv', chunksize=chunksize)

dfs = []
total_rows = 0

for chunk in chunks:
    total_rows += len(chunk)
    dfs.append(chunk)

df = pd.concat(dfs, ignore_index=True) 

print(total_rows)


6754200


In [11]:
df.head(5)

,Transfer Date,Postcode,primary_addressable_object_name,secondary_addressable_object_name,street,Town/City,District,County,Price (Thousands),Freehold Tenure,...,Postcode Area,Postcode District,Area Code Average Price (Thousands),Town/City Average Price (Thousands),District Average Price (Thousands),County Average Price (Thousands),LAT,LONG,OSEAST1M,OSNRTH1M
0,2018-09-24,M6 8GQ,1,NaN,RIVINGTON ROAD,SALFORD,SALFORD,GREATER MANCHESTER,253.500,1,...,M,M6,286.373466,274.417259,259.866315,264.161265,53.49295,-2.311376,379439.0,399626.0
1,2018-09-28,WA3 2UE,35,NaN,STONEACRE CLOSE,WARRINGTON,WIGAN,GREATER MANCHESTER,231.950,1,...,WA,WA3,301.647233,295.472854,194.319863,264.161265,53.46955,-2.583977,361332.0,397136.0
2,2018-08-29,OL6 6RJ,102,NaN,THORNFIELD GROVE,ASHTON-UNDER-LYNE,TAMESIDE,GREATER MANCHESTER,112.500,1,...,OL,OL6,197.639388,184.326053,196.955406,264.161265,53.48984,-2.074338,395165.0,399237.0
3,2018-06-15,M46 0TW,37,NaN,THREADNEEDLE PLACE,MANCHESTER,WIGAN,GREATER MANCHESTER,184.995,1,...,M,M46,286.373466,288.415706,194.319863,264.161265,53.52589,-2.495181,367270.0,403360.0
4,2018-09-28,M28 3XS,9,NaN,MARPLE GARDENS,MANCHESTER,SALFORD,GREATER MANCHESTER,214.995,0,...,M,M28,286.373466,288.415706,259.866315,264.161265,53.51677,-2.411523,372810.0,402310.0


In [21]:
df['Postcode'] = df['Postcode'].str.replace(' ', '').str.upper()
df.head()

,Transfer Date,Postcode,primary_addressable_object_name,secondary_addressable_object_name,street,Town/City,District,County,Price (Thousands),Freehold Tenure,...,Postcode Area,Postcode District,Area Code Average Price (Thousands),Town/City Average Price (Thousands),District Average Price (Thousands),County Average Price (Thousands),LAT,LONG,OSEAST1M,OSNRTH1M
0,2018-09-24,M68GQ,1,NaN,RIVINGTON ROAD,SALFORD,SALFORD,GREATER MANCHESTER,253.500,1,...,M,M6,286.373466,274.417259,259.866315,264.161265,53.49295,-2.311376,379439.0,399626.0
1,2018-09-28,WA32UE,35,NaN,STONEACRE CLOSE,WARRINGTON,WIGAN,GREATER MANCHESTER,231.950,1,...,WA,WA3,301.647233,295.472854,194.319863,264.161265,53.46955,-2.583977,361332.0,397136.0
2,2018-08-29,OL66RJ,102,NaN,THORNFIELD GROVE,ASHTON-UNDER-LYNE,TAMESIDE,GREATER MANCHESTER,112.500,1,...,OL,OL6,197.639388,184.326053,196.955406,264.161265,53.48984,-2.074338,395165.0,399237.0
3,2018-06-15,M460TW,37,NaN,THREADNEEDLE PLACE,MANCHESTER,WIGAN,GREATER MANCHESTER,184.995,1,...,M,M46,286.373466,288.415706,194.319863,264.161265,53.52589,-2.495181,367270.0,403360.0
4,2018-09-28,M283XS,9,NaN,MARPLE GARDENS,MANCHESTER,SALFORD,GREATER MANCHESTER,214.995,0,...,M,M28,286.373466,288.415706,259.866315,264.161265,53.51677,-2.411523,372810.0,402310.0


In [28]:
import os
import pandas as pd
import re
from tqdm import tqdm

def normalize_address_components(*components):
    combined = ' '.join([str(c) for c in components if pd.notna(c)])
    normalized = (
        combined.upper()
        .replace(',', '')
        .replace('.', '')
        .replace('-', ' ')
    )
    normalized = re.sub(r'\s+', ' ', normalized)
    return normalized.strip()

df['postcode_clean'] = df['Postcode'].astype(str).str.replace(' ', '').str.upper()
df['match_key'] = df.apply(
    lambda row: normalize_address_components(
        row.get('primary_addressable_object_name', ''),
        row.get('street', ''),
        row['postcode_clean']
    ),
    axis=1
)

folder = 'epc_certificates_only'
subfolders = [f for f in os.listdir(folder) if os.path.isdir(os.path.join(folder, f))]

output_dir = "matched_epc_rows"
os.makedirs(output_dir, exist_ok=True)

for subfolder in tqdm(subfolders, desc="Processing EPC folders", unit="file"):
    cert_path = os.path.join(folder, subfolder, 'certificates.csv')

    if not os.path.exists(cert_path):
        continue

    try:
        epc_df = pd.read_csv(cert_path, low_memory=False)

        epc_df['postcode_clean'] = epc_df['POSTCODE'].astype(str).str.replace(' ', '').str.upper()
        epc_df['match_key'] = epc_df.apply(
            lambda row: normalize_address_components(
                row.get('ADDRESS1', ''),
                row.get('ADDRESS2', ''),
                row['postcode_clean']
            ),
            axis=1
        )

        matched = epc_df[epc_df['match_key'].isin(df['match_key'])]
        print('Done')

        if not matched.empty:
            out_path = os.path.join(output_dir, f"{subfolder}.parquet")
            matched.to_parquet(out_path, index=False)

    except Exception as e:
        print(f"❌ Error processing {cert_path}: {e}")

Processing EPC folders:   0%|          | 0/347 [00:00<?, ?file/s]

Done


Processing EPC folders:   0%|          | 1/347 [00:05<29:41,  5.15s/file]

Done


Processing EPC folders:   1%|          | 2/347 [00:12<36:22,  6.33s/file]

...

Done


Processing EPC folders: 100%|█████████▉| 346/347 [37:28<00:04,  4.74s/file]

Done


Processing EPC folders: 100%|██████████| 347/347 [37:30<00:00,  6.48s/file]

Done


In [29]:
import os
import pandas as pd

matched_dir = 'matched_epc_rows'

parquet_files = [
    os.path.join(matched_dir, f)
    for f in os.listdir(matched_dir)
    if f.endswith('.parquet')
]

matched_dfs = [pd.read_parquet(f) for f in parquet_files]

if matched_dfs:
    total_matched = pd.concat(matched_dfs, ignore_index=True)
    print(f"✅ Loaded total matched rows: {len(total_matched):,}")
    display(total_matched.head())
else:
    print("⚠ No parquet files found in matched_epc_rows.")

✅ Loaded total matched rows: 4,178,789


,LMK_KEY,ADDRESS1,ADDRESS2,ADDRESS3,POSTCODE,BUILDING_REFERENCE_NUMBER,CURRENT_ENERGY_RATING,POTENTIAL_ENERGY_RATING,CURRENT_ENERGY_EFFICIENCY,POTENTIAL_ENERGY_EFFICIENCY,...,CONSTRUCTION_AGE_BAND,LODGEMENT_DATETIME,TENURE,FIXED_LIGHTING_OUTLETS_COUNT,LOW_ENERGY_FIXED_LIGHT_COUNT,UPRN,UPRN_SOURCE,REPORT_TYPE,postcode_clean,match_key
0,1451865729102016060917325243560118,"54, Chatham Road",None,None,TS24 8QQ,134825478,D,B,61,85,...,England and Wales: 1930-1949,2016-06-09 17:32:52,owner-occupied,NaN,NaN,1.001100e+11,Address Matched,100,TS248QQ,54 CHATHAM ROAD TS248QQ
1,978835396212019052007505697910817,"19, Wainwright Walk",None,None,TS25 1XA,8777671178,D,C,65,79,...,England and Wales: 1967-1975,2019-05-20 07:50:56,owner-occupied,NaN,NaN,1.001100e+11,Address Matched,100,TS251XA,19 WAINWRIGHT WALK TS251XA
2,1341712688132019081910310554978408,"116, Milton Road",None,None,TS26 8DX,8864447378,D,C,56,77,...,England and Wales: 1900-1929,2019-08-19 10:31:05,rental (private),NaN,NaN,1.001100e+11,Address Matched,100,TS268DX,116 MILTON ROAD TS268DX
3,1186183659802014081111373821640768,"18, Brafferton Street",None,None,TS26 8LH,4895056278,D,B,57,90,...,England and Wales: 1900-1929,2014-08-11 11:37:38,rental (private),7.0,0.0,1.001100e+11,Address Matched,100,TS268LH,18 BRAFFERTON STREET TS268LH
4,1566486539802017081008512757330418,"132, Kesteven Road",None,None,TS25 2NJ,862833578,C,B,70,85,...,England and Wales: 1967-1975,2017-08-10 08:51:27,owner-occupied,NaN,NaN,1.001100e+11,Address Matched,100,TS252NJ,132 KESTEVEN ROAD TS252NJ


In [31]:
dfmax = total_matched[total_matched['CURRENT_ENERGY_EFFICIENCY'] > 100]
dfmax.shape

(1772, 95)

In [40]:
dfmax = dfmax.sort_values('CURRENT_ENERGY_EFFICIENCY')
dfmax.tail()

,LMK_KEY,ADDRESS1,ADDRESS2,ADDRESS3,POSTCODE,BUILDING_REFERENCE_NUMBER,CURRENT_ENERGY_RATING,POTENTIAL_ENERGY_RATING,CURRENT_ENERGY_EFFICIENCY,POTENTIAL_ENERGY_EFFICIENCY,...,CONSTRUCTION_AGE_BAND,LODGEMENT_DATETIME,TENURE,FIXED_LIGHTING_OUTLETS_COUNT,LOW_ENERGY_FIXED_LIGHT_COUNT,UPRN,UPRN_SOURCE,REPORT_TYPE,postcode_clean,match_key
2727841,801830106732013090919575667268109,"69, Monton Street",None,None,M14 4LS,967139968,A,A,143,144,...,England and Wales: 2007 onwards,2013-09-09 19:57:56,rental (private),2.0,2.0,7.711577e+07,Address Matched,100,M144LS,69 MONTON STREET M144LS
1099044,130401e5aa8581948660c7249f577d77ffdb27a14e5694...,Hatchlands,Rectory Lane,Charlton Musgrove,BA9 8ET,5066324278,A,A,144,158,...,England and Wales: 1967-1975,2024-05-17 06:00:58,Owner-occupied,30.0,NaN,3.000074e+07,Energy Assessor,100,BA98ET,HATCHLANDS RECTORY LANE BA98ET
2002614,1134005089702019013119395721210628,The Island,Yarmouth Road,None,NR7 0HE,3752382278,A,A,157,170,...,England and Wales: 1950-1966,2019-01-31 19:39:57,owner-occupied,NaN,NaN,2.000043e+11,Address Matched,100,NR70HE,THE ISLAND YARMOUTH ROAD NR70HE
1463833,688824189222013102413461480348167,"14, Gainsborough Close",None,None,CO16 8DW,6812880968,A,A,161,166,...,England and Wales: 1983-1990,2013-10-24 13:46:14,owner-occupied,25.0,20.0,1.000906e+11,Address Matched,100,CO168DW,14 GAINSBOROUGH CLOSE CO168DW
1349245,910920099502013041714444003679368,High Cross Hall,Station Road,Rotherfield,TN6 3HR,9554896078,A,A,165,165,...,NO DATA!,2013-04-17 14:44:40,NO DATA!,52.0,52.0,1.002438e+10,Address Matched,101,TN63HR,HIGH CROSS HALL STATION ROAD TN63HR


In [45]:
unique_vals = total_matched['BUILDING_REFERENCE_NUMBER'].unique()
Unique_df = total_matched[total_matched['BUILDING_REFERENCE_NUMBER'].isin(unique_vals)]
Unique_df.shape

(4178789, 95)

In [46]:
df.columns

Index(['Transfer Date', 'Postcode', 'primary_addressable_object_name',
       'secondary_addressable_object_name', 'street', 'Town/City', 'District',
       'County', 'Price (Thousands)', 'Freehold Tenure', 'Leasehold Tenure',
       'Old Build', 'New Build', 'Detached', 'Flat', 'Other Property Type',
       'Semi-detached', 'Terraced', 'Year', 'Month', 'Quarter',
       'Day of the Week', 'Postcode Area', 'Postcode District',
       'Area Code Average Price (Thousands)',
       'Town/City Average Price (Thousands)',
       'District Average Price (Thousands)',
       'County Average Price (Thousands)', 'LAT', 'LONG', 'OSEAST1M',
       'OSNRTH1M', 'postcode_clean', 'match_key'],
      dtype='object')

In [47]:
trim_price = df.drop(columns=['primary_addressable_object_name', 'secondary_addressable_object_name', 'street', 'District', 'Town/City'], axis=1)
trim_price.columns

Index(['Transfer Date', 'Postcode', 'County', 'Price (Thousands)',
       'Freehold Tenure', 'Leasehold Tenure', 'Old Build', 'New Build',
       'Detached', 'Flat', 'Other Property Type', 'Semi-detached', 'Terraced',
       'Year', 'Month', 'Quarter', 'Day of the Week', 'Postcode Area',
       'Postcode District', 'Area Code Average Price (Thousands)',
       'Town/City Average Price (Thousands)',
       'District Average Price (Thousands)',
       'County Average Price (Thousands)', 'LAT', 'LONG', 'OSEAST1M',
       'OSNRTH1M', 'postcode_clean', 'match_key'],
      dtype='object')

In [48]:
 total_matched.columns

Index(['LMK_KEY', 'ADDRESS1', 'ADDRESS2', 'ADDRESS3', 'POSTCODE',
       'BUILDING_REFERENCE_NUMBER', 'CURRENT_ENERGY_RATING',
       'POTENTIAL_ENERGY_RATING', 'CURRENT_ENERGY_EFFICIENCY',
       'POTENTIAL_ENERGY_EFFICIENCY', 'PROPERTY_TYPE', 'BUILT_FORM',
       'INSPECTION_DATE', 'LOCAL_AUTHORITY', 'CONSTITUENCY', 'COUNTY',
       'LODGEMENT_DATE', 'TRANSACTION_TYPE', 'ENVIRONMENT_IMPACT_CURRENT',
       'ENVIRONMENT_IMPACT_POTENTIAL', 'ENERGY_CONSUMPTION_CURRENT',
       'ENERGY_CONSUMPTION_POTENTIAL', 'CO2_EMISSIONS_CURRENT',
       'CO2_EMISS_CURR_PER_FLOOR_AREA', 'CO2_EMISSIONS_POTENTIAL',
       'LIGHTING_COST_CURRENT', 'LIGHTING_COST_POTENTIAL',
       'HEATING_COST_CURRENT', 'HEATING_COST_POTENTIAL',
       'HOT_WATER_COST_CURRENT', 'HOT_WATER_COST_POTENTIAL',
       'TOTAL_FLOOR_AREA', 'ENERGY_TARIFF', 'MAINS_GAS_FLAG', 'FLOOR_LEVEL',
       'FLAT_TOP_STOREY', 'FLAT_STOREY_COUNT', 'MAIN_HEATING_CONTROLS',
       'MULTI_GLAZE_PROPORTION', 'GLAZED_TYPE', 'GLAZED_AREA',
      

In [49]:
matched_trimmed = total_matched.drop(
    columns=[
        'LMK_KEY', 'ADDRESS3', 'BUILDING_REFERENCE_NUMBER', 'INSPECTION_DATE',
        'LOCAL_AUTHORITY', 'CONSTITUENCY', 'COUNTY', 'LODGEMENT_DATE',
        'TRANSACTION_TYPE', 'ENVIRONMENT_IMPACT_CURRENT', 'ENVIRONMENT_IMPACT_POTENTIAL',
        'ENERGY_CONSUMPTION_CURRENT', 'ENERGY_CONSUMPTION_POTENTIAL',
        'CO2_EMISSIONS_CURRENT', 'CO2_EMISS_CURR_PER_FLOOR_AREA',
        'CO2_EMISSIONS_POTENTIAL', 'LIGHTING_COST_CURRENT', 'LIGHTING_COST_POTENTIAL',
        'HEATING_COST_CURRENT', 'HEATING_COST_POTENTIAL', 'HOT_WATER_COST_CURRENT',
        'HOT_WATER_COST_POTENTIAL', 'ENERGY_TARIFF', 'FLOOR_DESCRIPTION',
        'WINDOWS_DESCRIPTION', 'WALLS_DESCRIPTION', 'SECONDHEAT_DESCRIPTION',
        'ROOF_DESCRIPTION', 'MAINHEAT_DESCRIPTION', 'MAINHEATCONT_DESCRIPTION',
        'LIGHTING_DESCRIPTION', 'HEAT_LOSS_CORRIDOR', 'UNHEATED_CORRIDOR_LENGTH',
        'PHOTO_SUPPLY', 'MECHANICAL_VENTILATION', 'ADDRESS', 'LOCAL_AUTHORITY_LABEL',
        'CONSTITUENCY_LABEL', 'POSTTOWN', 'LODGEMENT_DATETIME', 'TENURE',
        'FIXED_LIGHTING_OUTLETS_COUNT', 'LOW_ENERGY_FIXED_LIGHT_COUNT',
        'UPRN', 'UPRN_SOURCE', 'REPORT_TYPE', 'postcode_clean'
    ]
)

In [50]:
merge_df = matched_trimmed.merge(trim_price, how='left', on=['match_key'])

In [51]:
merge_df.shape

(4824809, 76)

In [52]:
merge_df.to_csv('merge.csv', index=False)

In [53]:
summary_df = pd.DataFrame({
    'Column': merge_df.columns,
    'Dtype': merge_df.dtypes.values,
    'Missing': merge_df.isna().sum().values,
    'Percent': (((merge_df.isna().sum().values)/4864524)*100).round(1)
})

print(summary_df)

                     Column    Dtype  Missing  Percent
0                  ADDRESS1   object        1      0.0
1                  ADDRESS2   object  4506651     92.6
2                  POSTCODE   object        0      0.0
3     CURRENT_ENERGY_RATING   object        0      0.0
4   POTENTIAL_ENERGY_RATING   object        0      0.0
..                      ...      ...      ...      ...
71                      LAT  float64       21      0.0
72                     LONG  float64       21      0.0
73                 OSEAST1M  float64       21      0.0
74                 OSNRTH1M  float64       21      0.0
75           postcode_clean   object        0      0.0

[76 rows x 4 columns]


In [54]:
summary_df.to_csv('column_summary.csv', index=False)

In [55]:
save=merge_df

In [56]:
drop_cols = ['ADDRESS2', 'FLAT_TOP_STOREY', 'FLAT_STOREY_COUNT','HOT_WATER_ENV_EFF', 'FLOOR_ENERGY_EFF', 'WALLS_ENV_EFF', 'SHEATING_ENERGY_EFF','FLOOR_ENV_EFF','SHEATING_ENV_EFF']
merge_df.drop(columns=drop_cols, inplace=True, errors='ignore')

mode_impute_cols = ['MAINS_GAS_FLAG', 'MAIN_HEATING_CONTROLS','GLAZED_TYPE', 'LOW_ENERGY_LIGHTING','SOLAR_WATER_HEATING_FLAG', 'MAIN_FUEL', 'GLAZED_AREA', 'ROOF_ENERGY_EFF', 'ROOF_ENV_EFF']

for col in mode_impute_cols:
    if col in merge_df.columns:
        mode_series = merge_df[col].mode(dropna=True)
        if not mode_series.empty:
            merge_df[col].fillna(mode_series[0], inplace=True)

median_impute_cols = ['MULTI_GLAZE_PROPORTION', '', 'EXTENSION_COUNT', 'NUMBER_HABITABLE_ROOMS', 'NUMBER_HEATED_ROOMS', 'FLOOR_HEIGHT']

for col in median_impute_cols:
    if col in merge_df.columns:
        merge_df[col].fillna(merge_df[col].median(), inplace=True)

zero_fill_cols = ['EXTENSION_COUNT', 'WIND_TURBINE_COUNT','NUMBER_OPEN_FIREPLACES']
for col in zero_fill_cols:
    if col in merge_df.columns:
        merge_df[col].fillna(0, inplace=True)

unknown_bin_cols = ['FLOOR_LEVEL']
for col in unknown_bin_cols:
    if col in merge_df.columns:
        merge_df[col] = merge_df[col].fillna('Unknown')

critical_cols = ['BUILT_FORM', 'LAT', 'LONG', 'OSEAST1M', 'OSNRTH1M','LIGHTING_ENV_EFF', 'LIGHTING_ENERGY_EFF', 'MAINHEATC_ENV_EFF','MAINHEATC_ENERGY_EFF', 'MAINHEAT_ENV_EFF', 'MAINHEAT_ENERGY_EFF','WALLS_ENERGY_EFF',  'WINDOWS_ENERGY_EFF','WINDOWS_ENV_EFF', 'HOT_WATER_ENERGY_EFF', 'HOTWATER_DESCRIPTION', 'ADDRESS1']

merge_df.dropna(subset=[col for col in critical_cols if col in merge_df.columns], inplace=True)

missing_summary = merge_df.isnull().sum()
print("Remaining missing values:\n", missing_summary[missing_summary > 0])

C:\Users\lenovo\AppData\Local\Temp\ipykernel_9976\3156588321.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merge_df[col].fillna(mode_series[0], inplace=True)
C:\Users\lenovo\AppData\Local\Temp\ipykernel_9976\3156588321.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For

Remaining missing values:
 CONSTRUCTION_AGE_BAND    43125
dtype: int64


In [57]:
merge_df['CONSTRUCTION_AGE_BAND'] = merge_df['CONSTRUCTION_AGE_BAND'].fillna('Unknown')
merge_df.shape

(4818318, 67)

In [58]:
turn_binary = merge_df
turn_binary.shape

(4818318, 67)

In [59]:
summary_df1 = pd.DataFrame({
    'Column': turn_binary.columns,
    'Dtype': turn_binary.dtypes.values,
    'Example': turn_binary.iloc[0].values

})

summary_df1.to_csv('dtype_inq.csv', index=False)

In [60]:
turn_binary = turn_binary.drop(columns=[
    'ADDRESS1','POSTCODE','CURRENT_ENERGY_RATING','POTENTIAL_ENERGY_RATING',
    'BUILT_FORM','FLOOR_LEVEL','GLAZED_TYPE','GLAZED_AREA',
    'MAIN_FUEL','PROPERTY_TYPE','match_key','County',
    'postcode_clean','Postcode District','Postcode Area'
])

turn_binary['MAINS_GAS_FLAG'] = turn_binary['MAINS_GAS_FLAG'].map({'Y': 1, 'N': 0})
turn_binary['SOLAR_WATER_HEATING_FLAG'] = turn_binary['SOLAR_WATER_HEATING_FLAG'].map({'Y': 1, 'N': 0})

scale = ['HOT_WATER_ENERGY_EFF', 'WINDOWS_ENERGY_EFF','WINDOWS_ENV_EFF', 'WALLS_ENERGY_EFF', 'ROOF_ENERGY_EFF','ROOF_ENV_EFF', 'MAINHEAT_ENERGY_EFF', 'MAINHEAT_ENV_EFF','MAINHEATC_ENERGY_EFF', 'MAINHEATC_ENV_EFF', 'LIGHTING_ENERGY_EFF','LIGHTING_ENV_EFF']

for i in scale:
    turn_binary[i] = turn_binary[i].map({'Very Poor': 0, 'Poor': 1, 'Average': 2, 'Good': 3,'Very Good': 4})

In [62]:
print(turn_binary['CONSTRUCTION_AGE_BAND'].unique())

['England and Wales: 1930-1949' 'England and Wales: 1967-1975'
 'England and Wales: 1900-1929' 'NO DATA!' 'England and Wales: 1950-1966'
 'England and Wales: before 1900' 'England and Wales: 2007 onwards'
 'England and Wales: 1983-1990' 'England and Wales: 1976-1982' 'INVALID!'
 '2019' 'England and Wales: 1996-2002' 'England and Wales: 2003-2006'
 '2020' 'England and Wales: 1991-1995' 'Unknown' '2021'
 'England and Wales: 2007-2011' '2017' '2022'
 'England and Wales: 2012 onwards' '2023' '2024' '2016' '2025'
 'England and Wales: 2012-2021' '2018' '2014' '2015' '2012' '1950' '1965'
 'England and Wales: 2022 onwards' '1783' '1920' '1900' '2008' '2002'
 '1999' '1800' '2009' '1970' '2010' '1890' '2011' '1889' '2013' '1907'
 '2007' '2000' '1888' '1960' '1930' '1935' '1850' '1947' '1975' '2031'
 '1958' '1910' '1881' '1700' '2033' '2006' '1967' '1940' '1954' '1820'
 '2005' '1915' '1927' '2003' '2042' '1983' '1905' '1870']


In [64]:
import numpy as np

def clean_age_band(val):
    if pd.isna(val):
        return np.nan
    val = str(val).strip()

    if val.startswith('England and Wales:'):
        val = val.replace('England and Wales:', '').strip()

    if val.isdigit():
        year = int(val)
        if year <= 1929:
            return '1900-1929'
        elif year <= 1949:
            return '1930-1949'
        elif year <= 1966:
            return '1950-1966'
        elif year <= 1975:
            return '1967-1975'
        elif year <= 1982:
            return '1976-1982'
        elif year <= 1990:
            return '1983-1990'
        elif year <= 1995:
            return '1991-1995'
        elif year <= 2002:
            return '1996-2002'
        elif year <= 2006:
            return '2003-2006'
        elif year <= 2011:
            return '2007-2011'
        elif year <= 2021:
            return '2012-2021'
        else:
            return '2022 onwards'

    if val in ['NO DATA!', 'INVALID!', 'None']:
        return np.nan

    return val

turn_binary['CONSTRUCTION_AGE_BAND_CLEAN'] = turn_binary['CONSTRUCTION_AGE_BAND'].apply(clean_age_band)

age_band_map = {
    'before 1900': 0,
    '1900-1929': 1,
    '1930-1949': 2,
    '1950-1966': 3,
    '1967-1975': 4,
    '1976-1982': 5,
    '1983-1990': 6,
    '1991-1995': 7,
    '1996-2002': 8,
    '2003-2006': 9,
    '2007-2011': 10,
    '2012-2021': 11,
    '2022 onwards': 12
}

turn_binary['CONSTRUCTION_AGE_BAND_ENCODED'] = turn_binary['CONSTRUCTION_AGE_BAND_CLEAN'].map(age_band_map)

In [65]:
turn_binary.dtypes

CURRENT_ENERGY_EFFICIENCY                int64
POTENTIAL_ENERGY_EFFICIENCY              int64
TOTAL_FLOOR_AREA                       float64
MAINS_GAS_FLAG                           int64
MAIN_HEATING_CONTROLS                   object
MULTI_GLAZE_PROPORTION                 float64
EXTENSION_COUNT                        float64
NUMBER_HABITABLE_ROOMS                 float64
NUMBER_HEATED_ROOMS                    float64
LOW_ENERGY_LIGHTING                    float64
NUMBER_OPEN_FIREPLACES                 float64
HOTWATER_DESCRIPTION                    object
HOT_WATER_ENERGY_EFF                     int64
WINDOWS_ENERGY_EFF                       int64
WINDOWS_ENV_EFF                          int64
WALLS_ENERGY_EFF                         int64
ROOF_ENERGY_EFF                          int64
ROOF_ENV_EFF                             int64
MAINHEAT_ENERGY_EFF                      int64
MAINHEAT_ENV_EFF                         int64
MAINHEATC_ENERGY_EFF                     int64
MAINHEATC_ENV

In [66]:
conv = turn_binary[turn_binary['MAIN_HEATING_CONTROLS'] != '%%MAINHEATCONTROL%%']

In [67]:
conv['MAIN_HEATING_CONTROLS'] = conv['MAIN_HEATING_CONTROLS'].astype('float64')

C:\Users\lenovo\AppData\Local\Temp\ipykernel_9976\3129212291.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  conv['MAIN_HEATING_CONTROLS'] = conv['MAIN_HEATING_CONTROLS'].astype('float64')


In [68]:
conv.dtypes

CURRENT_ENERGY_EFFICIENCY                int64
POTENTIAL_ENERGY_EFFICIENCY              int64
TOTAL_FLOOR_AREA                       float64
MAINS_GAS_FLAG                           int64
MAIN_HEATING_CONTROLS                  float64
MULTI_GLAZE_PROPORTION                 float64
EXTENSION_COUNT                        float64
NUMBER_HABITABLE_ROOMS                 float64
NUMBER_HEATED_ROOMS                    float64
LOW_ENERGY_LIGHTING                    float64
NUMBER_OPEN_FIREPLACES                 float64
HOTWATER_DESCRIPTION                    object
HOT_WATER_ENERGY_EFF                     int64
WINDOWS_ENERGY_EFF                       int64
WINDOWS_ENV_EFF                          int64
WALLS_ENERGY_EFF                         int64
ROOF_ENERGY_EFF                          int64
ROOF_ENV_EFF                             int64
MAINHEAT_ENERGY_EFF                      int64
MAINHEAT_ENV_EFF                         int64
MAINHEATC_ENERGY_EFF                     int64
MAINHEATC_ENV

In [69]:
conv = conv.drop(columns='HOTWATER_DESCRIPTION')

In [70]:
conv = conv.drop(columns=['CONSTRUCTION_AGE_BAND_CLEAN','CONSTRUCTION_AGE_BAND'])

In [71]:
conv.shape

(4818234, 51)

In [72]:
epc_rename_map = {
    'CURRENT_ENERGY_EFFICIENCY': 'Current Energy Efficiency (EPC Score)',
    'POTENTIAL_ENERGY_EFFICIENCY': 'Potential Energy Efficiency (EPC Score)',
    'TOTAL_FLOOR_AREA': 'Total Floor Area (m²)',
    'MAINS_GAS_FLAG': 'Has Mains Gas',
    'MAIN_HEATING_CONTROLS': 'Main Heating Controls',
    'MULTI_GLAZE_PROPORTION': 'Proportion of Multi-Glazing (%)',
    'EXTENSION_COUNT': 'Number of Extensions',
    'NUMBER_HABITABLE_ROOMS': 'Number of Habitable Rooms',
    'NUMBER_HEATED_ROOMS': 'Number of Heated Rooms',
    'LOW_ENERGY_LIGHTING': 'Low Energy Lighting (%)',
    'NUMBER_OPEN_FIREPLACES': 'Number of Open Fireplaces',
    'HOT_WATER_ENERGY_EFF': 'Hot Water Energy Efficiency',
    'WINDOWS_ENERGY_EFF': 'Windows Energy Efficiency',
    'WINDOWS_ENV_EFF': 'Windows Environmental Impact',
    'WALLS_ENERGY_EFF': 'Walls Energy Efficiency',
    'ROOF_ENERGY_EFF': 'Roof Energy Efficiency',
    'ROOF_ENV_EFF': 'Roof Environmental Impact',
    'MAINHEAT_ENERGY_EFF': 'Main Heating Energy Efficiency',
    'MAINHEAT_ENV_EFF': 'Main Heating Environmental Impact',
    'MAINHEATC_ENERGY_EFF': 'Main Heating Controls Energy Efficiency',
    'MAINHEATC_ENV_EFF': 'Main Heating Controls Environmental Impact',
    'LIGHTING_ENERGY_EFF': 'Lighting Energy Efficiency',
    'LIGHTING_ENV_EFF': 'Lighting Environmental Impact',
    'WIND_TURBINE_COUNT': 'Number of Wind Turbines',
    'FLOOR_HEIGHT': 'Floor Height (m)',
    'SOLAR_WATER_HEATING_FLAG': 'Has Solar Water Heating'
}
conv = conv.rename(columns=epc_rename_map)

In [74]:
conv = conv.drop(columns=['Transfer Date','Postcode'])

In [ ]:
conv.to_csv('EPC_Fin.csv', index=False)